# S3 수명 주기
- 일정 기간 이후 오브젝트의 상태를 변경 가능
    - 전환 작업(Transition actions) : 오브젝트를 다른 스토리지 티어로 옮기기
        - 예 : 특정 파일을 30일 후 아카이브로 보낸다
        - 예 : 50일 후 객체를 IA로 보낸다
    - 만료 작업(Expiration actions) : 오브젝트를 삭제
- 수명 주기 설정 시 소급 적용
    - 예 : 30일 이후 만료 작업을 설정했다면, 기존 버킷의 모든 30일 이상된 오브젝트에도 같이 적용
- 필터 및 날짜 적용 가능
    - 예 : 특정 경로에 있는 5mb이상 파일을 30일 뒤에 모두 아카이브한다
    - 예 : 2024-05-30일부터 특정 디렉토리에 있는 이미지를 모두 삭제한다

## 전환 작업
- 일정 기간 이후 오브젝트의 클래스를 변경 가능
    - Waterfall 모델 사용 : 스탠다드에서는 ia, glacier등 하위 티어로 이동 가능 // IA에서 스탠다드로 상위 티어 이동은 불가능
- 제약 사항 있음
    - 예 : 다른 클래스에서 스탠다드로 변경 불가능
    - 128kb 미만 파일의 경우, Standard/IA 에서 Glacier Instant Retrieval로 이동 불가능


## 만료 작업
- 버저닝에 따라 다른 방식
    - 버저닝이 없을 때 : async로 영구 삭제 (해당 객체가 즉시 삭제된 것 처럼 보이지만, 실제로는 버킷 안에서 삭제가 진행 중)
    - 버저닝이 있을 때 : delete marker가 현재 version이 아니라면, marker 추가 후 현재 버전으로 격상
        - 현재 버전이 delete marker라면 동작하지 않음
- 삭제 날짜에 바로 삭제되지 않고 일정 딜레이 발생 가능
    - 그 동안은 비용 없음
    - HeadObject / GetObject API로 삭제 예정 확인 가능
- UTC 기준 정각에 수행되며 첫 번째 Rule은 약 48시간 정도 소요 가능
- 실패한 멀티파트 업로드 파일 삭제 설정도 여기서 가능

## 수명 주기의 조건
- 객체의 수명
    - 전환 혹은 생성부터 일정 기간 경과 후 효과 발생
    - 예 : 전환 후 30일이 지나면 또 다른 티어로 전환
- 날짜
    - 특정 날짜부터 효과 발생, 단 콘솔로 설정 불가능 (확인만 가능)
    - 예 : 5/30부터 모든 오브젝트 삭제 : 이 경우 5월 30일 이후 다른 오브젝트가 추가되어도 바로 삭제됨
- 필터
    - 두 조건 모두 필터링의 조건을 만족해야 발동


## 필터
- 특정 조건으로 필터링 가능
    - Prefix : /log, /log/production 등 디렉토리로 설정 가능
    - tag : 특정 태그를 대상으로 설정 가능, and 로 연결 가능
    - 오브젝트 사이즈 : 1mb이상 같은 조건 설정 가능
- 필터링 조건 조합 가능
    - 예 : /image prefix & 5mb이상